# Libraries

In [1]:
import numpy as np
import pandas as pd


from pkg.constants    import *
from pkg.evaluation   import Evaluation
from pkg.algorithims  import *
from pkg.neighborhood import *

from time import time
from math import exp

# Simulates Annealing

In [2]:
seeds = np.array([7054, 1354, 23503, 11268, 58283])

ev = Evaluation()

granularity = 2 # estudiado en local search


A continuación se muestran los parámetros que quedan por ajustar y que ajustaremos en las siguientes celdas

In [3]:
def run_simulated_an(seeds, L, k_max, mu, fi, granularity, ev):
    costs       = np.array([])
    evaluations = np.array([])

    ini = time()

    greedy_cost = 574.5513854177129
    t0 = mu * greedy_cost / (- np.log(fi))
    tf = t0 / (1 + k_max)

    for s in seeds:

        # set evaluation calls counter to 0
        ev.total_calls = 0

        np.random.seed(s)

        solution, cost = simulated_an(
            t0         = t0,
            tf         = tf,
            L          = L, 
            slots      = granularity, 
            evaluation = ev
        )

        costs = np.append(costs, cost)
        evaluations = np.append(evaluations, ev.total_calls)


    fin = time()

    df = pd.DataFrame({
        "L"                 : [L],
        "k_max"             : [k_max],
        "mu"                : [mu],
        "fi"                : [fi],
        "tiempo_ejecucion"  : [fin-ini],
        "media_coste"       : [costs.mean()],
        "std_coste"         : [costs.std()],
        "media_evaluaciones": [evaluations.mean()],
        "std_evaluaciones"  : [evaluations.std()],
        "coste_mejor"       : [costs.min()]
    })

    return df


In [4]:
# k_max: maximo numero iteraciones: en el entorno de 80
# condicion de parada
k_max = 80

# L(T): condicion enfriamiento. Numero fijo para cada iteracion
# Se enfria la temperatura al final de una iteracion
# La iteracion finaliza al visitar todos los vecinos
L = 20

# mu: tanto por uno [0,1], indica la probabilidad de aceptar una solucion peor que la inicial
mu = 0.2

# fi: probabilidad de aceptar una solucion un mu por 1 peor que la inicial
fi = 0.2

df = run_simulated_an(
    seeds       = seeds,
    L           = L,
    k_max       = k_max,
    mu          = mu,
    fi          = fi,
    granularity = granularity,
    ev          = ev
)


1.03151437484382
1.1476436882635743
1.195532685201504
0.8567804702660197
0.9684198821244381
0.8988841519597965
0.693375429868042
1.0699977739832942
1.0495145650672182
1.0459909368618268
1.0994363859181737
0.9631534356751613
0.8443134322179877
0.8756752661052318
0.8848337788501567
0.997307247813386
0.8620449515822586
0.7443781312127208
1.4096945008454465
1.1312464692126347
0.9837968299353819
0.9593753833034807
1.3473035079876967
0.752290784210783
0.9977374051854951
0.792061154245653
0.7173807806094208
0.9005925958720535
1.1768586557438934
1.1075815330925234
1.228639734421081
1.3436705296279878
0.9834646436260438
0.9475374616317733
1.0134837133803898
0.7834538087570466
0.9461722129791008
0.741399697585445
1.0671257442448612
1.0813085690183053
1.0525894420461148
0.8581240454248802
1.0626243433454927
1.0864828080443139
1.1841594537750924
1.8321427241905108
0.8567085757479833
0.9411392232957713
1.5954226578780242
0.9320564613839515
1.0612123406737584
0.5550553906511838
0.5963523318433922
0.

KeyboardInterrupt: 

In [15]:
df

,L,k_max,mu,fi,tiempo_ejecucion,media_coste,std_coste,media_evaluaciones,std_evaluaciones,coste_mejor
0,20,80,0.2,0.2,424.141814,388.57637,13.950863,1641.0,0.0,375.322033


## Ajuste de parametros

Ajuste de mu y fi. Guardamos todos los mu y fi que cumplan la condición en un dataframe 

In [30]:
greedy_solution = np.array([ 7,  9, 18,  8, 11, 18, 11, 12,  8, 13, 13, 24, 11, 18, 20, 19])

n = Neighborhood(greedy_solution,2)
candidata = n.get_neighbor()

cost_actual = ev.evaluate(greedy_solution)
cost_cand = ev.evaluate(candidata)
dif = cost_cand - cost_actual

while dif < 0:
    candidata = n.get_neighbor()
    cost_actual = ev.evaluate(greedy_solution)
    cost_cand = ev.evaluate(candidata)

    dif = cost_cand - cost_actual

mus = np.array([])
fis = np.array([])
t = 0


for mu in np.arange(0.1,0.32, 0.02):
    for fi in np.arange(0.1,0.32, 0.02):

        aceptadas = 0
        t +=1

        T = mu * cost_actual / (- np.log(fi))

        for it in range(1000):

            if np.random.rand() < exp(-dif/T):
                aceptadas += 1
        
        if aceptadas > 800:
            mus = np.append(mus,mu)
            fis = np.append(fis,fi)

df_mu_fi = pd.DataFrame({
    "mu": mus,
    "fi": fis
})

t

121

In [31]:
df_mu_fi


,mu,fi
0,0.22,0.28
1,0.22,0.30
2,0.24,0.26
3,0.24,0.30
4,0.26,0.24
5,0.26,0.26
6,0.26,0.28
7,0.26,0.30
8,0.28,0.20
9,0.28,0.22


In [33]:
a = np.random.randint(0,21)

In [34]:
# escogemos los valores de mu y fi:
df_mu_fi.iloc[a,:]

mu    0.26
fi    0.28
Name: 6, dtype: float64

In [4]:
mu = 0.26
fi = 0.28

Ajuste del numero de iteraciones (k_max) y numero de vecinos generados

In [12]:
k = np.array([70, 80, 90])
l = np.array([20, 30, 40])
mu = 0.26
fi = 0.28

df = pd.DataFrame({
        "L"                 : [0],
        "k_max"             : [0],
        "mu"                : [0],
        "fi"                : [0],
        "tiempo_ejecucion"  : [0],
        "media_coste"       : [0],
        "std_coste"         : [0],
        "media_evaluaciones": [0],
        "std_evaluaciones"  : [0],
        "coste_mejor"       : [0]
    })

for k_max in k:
    for L in l:
        df_aux = run_simulated_an(
            seeds       = seeds,
            L           = L,
            k_max       = k_max,
            mu          = mu,
            fi          = fi,
            granularity = granularity,
            ev          = ev
        )

        df = pd.concat( [df, df_aux] )

        df.to_excel('./01_stats/simulated_an' + str(k_max) +'_' + str(L)+  '.xlsx')

        print(f'{k_max} y {L} hecho')

KeyboardInterrupt: 